In [1]:
import torch

**tensor operations**

In [15]:
a = torch.tensor(1)
b = torch.tensor(2)

In [16]:
a

tensor(1)

In [17]:
a.shape

torch.Size([])

In [26]:
a = torch.tensor([[1, 2]])
b = torch.tensor([[1],[2]])

In [35]:
print(a.shape, b.shape)

torch.Size([1, 2]) torch.Size([2, 1])


In [36]:
b = torch.ones(1,2)

In [38]:
a+b

tensor([[2., 3.]])

In [39]:
a-b

tensor([[0., 1.]])

In [43]:
print(a*b) #element-wise multiplication
print(torch.mul(a,b))

tensor([[1., 2.]])
tensor([[1., 2.]])


In [44]:
torch.matmul(a,b) # matrix multiplication

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x2 and 1x2)

In [61]:
a = torch.tensor([[1,2]], dtype=torch.float) 
#changed datatype to avoid dtype error in matmul

In [64]:
r = torch.randn(2,5)
print(r)

tensor([[ 1.4389, -0.9760,  1.9807, -0.9520, -1.8536],
        [ 0.8924, -0.9110, -0.7668, -1.7203,  0.6885]])


**matrix multiplication, slicing, transpose, reshaping**

In [65]:
# (1,2) X (2, 5) => (1, 5)
c = torch.matmul(a, r)
print(c)

tensor([[ 3.2237, -2.7980,  0.4471, -4.3925, -0.4765]])


In [66]:
c.shape

torch.Size([1, 5])

In [76]:
print(c[0][1:3])
print(c[0][1:-1])

tensor([-2.7980,  0.4471])
tensor([-2.7980,  0.4471, -4.3925])


In [84]:
print(c[0][:]) # print entire row

tensor([ 3.2237, -2.7980,  0.4471, -4.3925, -0.4765])


In [91]:
transposed_r = torch.transpose(r, 0, 1) #swap rows(0) with cols(1)

In [92]:
transposed_r

tensor([[ 1.4389,  0.8924],
        [-0.9760, -0.9110],
        [ 1.9807, -0.7668],
        [-0.9520, -1.7203],
        [-1.8536,  0.6885]])

In [93]:
transposed_r.shape

torch.Size([5, 2])

In [94]:
x = torch.randn(1,12)

In [95]:
x

tensor([[ 0.1761, -0.1710, -0.5520,  0.5596, -0.0193, -1.8063,  1.1434, -0.0116,
          1.6118, -1.6359, -2.1352,  0.0660]])

In [96]:
y = x.view(2,6)
z = x.view(4,3)

In [98]:
print(y, y.shape)
print(z, z.shape)

tensor([[ 0.1761, -0.1710, -0.5520,  0.5596, -0.0193, -1.8063],
        [ 1.1434, -0.0116,  1.6118, -1.6359, -2.1352,  0.0660]]) torch.Size([2, 6])
tensor([[ 0.1761, -0.1710, -0.5520],
        [ 0.5596, -0.0193, -1.8063],
        [ 1.1434, -0.0116,  1.6118],
        [-1.6359, -2.1352,  0.0660]]) torch.Size([4, 3])


**Autograd for vector**

In [36]:
a = torch.tensor([1,1], dtype=torch.float, requires_grad = True)
b = torch.tensor([2,3], dtype=torch.float, requires_grad = True)

In [39]:
Q = 3*a**2 - 2*b # Assume it to be loss func
print(Q)

tensor([-1., -3.], grad_fn=<SubBackward0>)


In [41]:
# Now we want to find derivative of Q wrt to 'a' and 'b', but since 'a' and 'b' are vector we will need external_grad
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

In [42]:
a.grad

tensor([6., 6.])

In [43]:
b.grad

tensor([-2., -2.])

In [44]:
print(a.grad == 6*a)
print(b.grad == -2)

tensor([True, True])
tensor([True, True])


**Running Entire Trainin Loop in Pytorch for Linear Regression**

In [2]:
x = torch.tensor([[1],[2],[3],[4]], dtype=torch.float32)
y = torch.tensor([[2],[4],[6],[7]], dtype=torch.float32)
print(f'Shape of x : {x.shape}')
print(f'Shape of y : {y.shape}')

Shape of x : torch.Size([4, 1])
Shape of y : torch.Size([4, 1])


In [4]:
import torch.nn as nn

In [5]:
n_samples, n_features = x.shape
print(n_samples, n_features)

input_size = n_features 
output_size = 1 # may be because we want to predict a single value for each data-pt.


learning_rate = 0.01
n_iter = 200
loss = nn.MSELoss()

4 1


In [6]:
class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        
        #define layers
        self.lin = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return self.lin(x)

In [8]:
x_test = torch.tensor([[3]], dtype=torch.float)

In [9]:
model = LinearRegression(input_size, output_size)
print(f'Prediction before training: f(5) = {model(x_test).item():.3f}')

Prediction before training: f(5) = -0.361


In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range (n_iter):
    y_pred = model(x)
    l = loss(y, y_pred)
    l.backward() # dl/dw
    optimizer.step() # update weights
    optimizer.zero_grad() # reset accumulated gradients to 0
    if(epoch%10 == 9):
        [w, b] = model.parameters()
        print(f'epoch {epoch+1}: w = {w[0][0].item()}, loss = {l:.8f}')

epoch 10: w = 1.5435541868209839, loss = 1.20947492
epoch 20: w = 1.780914306640625, loss = 0.12718883
epoch 30: w = 1.8159600496292114, loss = 0.09785819
epoch 40: w = 1.8185561895370483, loss = 0.09584749
epoch 50: w = 1.8160226345062256, loss = 0.09461641
epoch 60: w = 1.812751293182373, loss = 0.09347420
epoch 70: w = 1.8094457387924194, loss = 0.09239890
epoch 80: w = 1.8062169551849365, loss = 0.09138618
epoch 90: w = 1.8030800819396973, loss = 0.09043242
epoch 100: w = 1.8000352382659912, loss = 0.08953423
epoch 110: w = 1.797080397605896, loss = 0.08868821
epoch 120: w = 1.794212818145752, loss = 0.08789146
epoch 130: w = 1.791429877281189, loss = 0.08714109
epoch 140: w = 1.7887290716171265, loss = 0.08643450
epoch 150: w = 1.786108136177063, loss = 0.08576886
epoch 160: w = 1.783564567565918, loss = 0.08514208
epoch 170: w = 1.7810962200164795, loss = 0.08455175
epoch 180: w = 1.7787007093429565, loss = 0.08399587
epoch 190: w = 1.7763758897781372, loss = 0.08347219
epoch 200

In [12]:
print(f'Prediction after training: f(3) = {model(x_test).item():.3f}')

Prediction after training: f(3) = 5.604
